In [4]:
import sys
sys.path.append("..")

import numpy as np
import pandas as pd
import MetaTrader5 as mt5

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sesto.constants import CURRENCY_PAIRS
from sesto.indicators import SMA, BB, RSI
import sesto.metatrader.data as mtd
from sesto.performance import performance

In [5]:
PAIRS = CURRENCY_PAIRS
TIMEFRAME = mt5.TIMEFRAME_H1
BARS = 3000
RSI_PERIOD = 14

# Configuration Variables
RSI_LOWER = 20
RSI_UPPER =80
INITIAL_CAPITAL = 10000
LEVERAGE = 10  # Leverage factor (e.g., 10x)
tp_p_sl_p_RATIO = 2  # Ratio between tp_price and sl_price (e.g., 2:1)
SPREAD_PERCENT = 0.012  # Overall spread between entry and tp_price/sl_price (e.g., 3%)
FEE_MULTIPLIER = 0.001  # Transaction fee as a percentage of the trade value (e.g., 0.1%)
CAPITAL_ALLOCATION_PER_TRADE = 0.1  # Allocate 10% of capital per trade

In [6]:
mtd.fill_data_pos(PAIRS, TIMEFRAME, BARS)

Exception fetching data for USDJPY on 16385: 'int' object has no attribute 'value'
No data fetched for pair: USDJPY on timeframe: 16385


KeyError: 16385

In [ ]:
for symbol, df in mtd.data.items():
    RSI(df, RSI_PERIOD)

mtd.data['AUDUSD']

In [ ]:
# Initialize an empty list to store trade details
trades = []

# Iterate through each symbol's data
for symbol, df in mtd.data.items():
    capital = INITIAL_CAPITAL
    open_positions = []

    for index, row in df.iterrows():
        rsi = row[f'rsi-{RSI_PERIOD}']
        close_price = row['close']
        trade_time = row['time']

        # Check for entry signals
        if rsi < RSI_LOWER or rsi > RSI_UPPER:
            if (rsi < RSI_LOWER and not any(pos['position'] == 'long' for pos in open_positions)) or \
               (rsi > RSI_UPPER and not any(pos['position'] == 'short' for pos in open_positions)):

                type = 'long' if rsi < RSI_LOWER else 'short'
                entry_price = close_price
                allocated_capital = capital * CAPITAL_ALLOCATION_PER_TRADE
                spread = entry_price * SPREAD_PERCENT
                tp_price = entry_price + spread if type == 'long' else entry_price - spread
                sl_price = entry_price - (spread / tp_p_sl_p_RATIO) if type == 'long' else entry_price + (spread / tp_p_sl_p_RATIO)
                
                # Calculate transaction fee when opening a position
                fee = FEE_MULTIPLIER * LEVERAGE * allocated_capital
                allocated_capital -= fee
                capital -= allocated_capital
                
                open_positions.append({
                    'symbol': symbol,
                    'entry_time': trade_time,
                    'entry_price': entry_price,
                    'sl_price': sl_price,
                    'tp_price': tp_price,
                    'position': type,
                    'allocated_capital': allocated_capital,
                    'fee': fee
                })

        # Check for exit signals
        for position in open_positions.copy():
            if position['position'] == 'long' and (rsi > RSI_UPPER or close_price >= position['tp_price'] or close_price <= position['sl_price']):
                profit = (close_price - position['entry_price']) * LEVERAGE * position['allocated_capital'] / position['entry_price']
                capital += profit - position['fee']
                max_drawdown = (position['entry_price'] - min(df.loc[index:, 'close'])) * LEVERAGE * position['allocated_capital'] / position['entry_price']
                
                trades.append({
                    'symbol': position['symbol'],
                    'entry_time': position['entry_time'],
                    'exit_time': trade_time,
                    'entry_price': position['entry_price'],
                    'close_price': close_price,
                    'pnl': profit - position['fee'],
                    'max_drawdown': max_drawdown,
                    'tp_price': position['tp_price'],
                    'sl_price': position['sl_price'],
                    'position': position['position']
                })
                open_positions.remove(position)
            
            elif position['position'] == 'short' and (rsi < RSI_LOWER or close_price <= position['tp_price'] or close_price >= position['sl_price']):
                profit = (position['entry_price'] - close_price) * LEVERAGE * position['allocated_capital'] / position['entry_price']
                capital += profit - position['fee']
                max_drawdown = (max(df.loc[index:, 'close']) - position['entry_price']) * LEVERAGE * position['allocated_capital'] / position['entry_price']
                
                trades.append({
                    'symbol': position['symbol'],
                    'entry_time': position['entry_time'],
                    'exit_time': trade_time,
                    'entry_price': position['entry_price'],
                    'close_price': close_price,
                    'pnl': profit - position['fee'],
                    'max_drawdown': max_drawdown,
                    'tp_price': position['tp_price'],
                    'sl_price': position['sl_price'],
                    'position': position['position']
                })
                open_positions.remove(position)

    # Ensure that any open positions are closed at the final market price
    final_row = df.iloc[-1]
    final_price = final_row['close']
    final_time = final_row['time']

    for position in open_positions:
        if position['position'] == 'long':
            profit = (final_price - position['entry_price']) * LEVERAGE * position['allocated_capital'] / position['entry_price']
        elif position['position'] == 'short':
            profit = (position['entry_price'] - final_price) * LEVERAGE * position['allocated_capital'] / position['entry_price']

        # Calculate transaction fee when closing a position
        fee = fee * LEVERAGE * position['allocated_capital']
        capital += profit - fee

        max_drawdown = (position['entry_price'] - min(df['close'])) * LEVERAGE * position['allocated_capital'] / position['entry_price'] if position['position'] == 'long' else (max(df['close']) - position['entry_price']) * LEVERAGE * position['allocated_capital'] / position['entry_price']
        
        trades.append({
            'symbol': position['symbol'],
            'entry_time': position['entry_time'],
            'exit_time': final_time,
            'entry_price': position['entry_price'],
            'close_price': final_price,
            'pnl': profit - fee,
            'max_drawdown': max_drawdown,
            'tp_price': position['tp_price'],
            'sl_price': position['sl_price'],
            'position': position['position']
        })

# Convert trades to a DataFrame for analysis
trades_df = pd.DataFrame(trades)
trades_df['capital'] = INITIAL_CAPITAL + trades_df['pnl'].cumsum()

trades_df

In [ ]:
performance(trades_df, INITIAL_CAPITAL)

In [ ]:


def plot_trades_for_all_symbols(trades_df, shared_data):
    # Define colors for different trade actions
    entry_color = '#2ca02c'  # Green for entry
    exit_color = '#d62728'   # Red for exit
    tp_p_color = '#1f77b4'     # Blue for Take Profit
    sl_p_color = '#ff7f0e'     # Orange for Stop Loss

    # Iterate through each symbol's data
    for symbol, price_data in shared_data.items():
        # Ensure time columns are in datetime format
        price_data['time'] = pd.to_datetime(price_data['time'])

        # Initialize figure with subplots (Price Chart and RSI)
        fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                            row_heights=[0.7, 0.3], vertical_spacing=0.1,
                            subplot_titles=(f'{symbol} Price Chart', 'RSI'))

        # Plot the price chart (legend appears only once)
        fig.add_trace(go.Scatter(x=price_data['time'], y=price_data['close'], 
                                 mode='lines', name=f'{symbol} Close Price', 
                                 line=dict(color='#1f77b4', width=2),
                                 showlegend=True), row=1, col=1)

        # Plot RSI if available (legend appears only once)
        if f'rsi-{RSI_PERIOD}' in price_data.columns:
            fig.add_trace(go.Scatter(x=price_data['time'], y=price_data[f'rsi-{RSI_PERIOD}'], 
                                     mode='lines', name='RSI', 
                                     line=dict(color='#ff7f0e', dash='dash'),
                                     showlegend=True), row=2, col=1)
            # Add RSI thresholds (these won't be in the legend)
            fig.add_hline(y=30, line=dict(color='red', dash='dash'), row=2, col=1)
            fig.add_hline(y=70, line=dict(color='green', dash='dash'), row=2, col=1)

        # Filter trades for the current symbol
        symbol_trades = trades_df[trades_df['symbol'] == symbol].copy()
        symbol_trades['entry_time'] = pd.to_datetime(symbol_trades['entry_time'])
        symbol_trades['exit_time'] = pd.to_datetime(symbol_trades['exit_time'])

        # Track whether tp_price and sl_price have been added to the legend
        tp_p_in_legend = False
        sl_p_in_legend = False

        # Plot each trade on the price chart
        for i, trade in symbol_trades.iterrows():
            entry_time = trade['entry_time']
            exit_time = trade['exit_time']
            entry_price = trade['entry_price']
            tp_price = trade['tp_price']
            sl_price = trade['sl_price']
            close_price = trade['close_price']
            pnl = trade['pnl']

            # Plot trade entry/exit as line with markers (only one legend entry for all trades)
            fig.add_trace(go.Scatter(
                x=[entry_time, exit_time], 
                y=[entry_price, close_price], 
                mode='lines+markers', 
                marker=dict(color=entry_color if pnl > 0 else exit_color, size=8),
                line=dict(color=entry_color if pnl > 0 else exit_color, width=2),
                name=f'Trade {"Profit" if pnl > 0 else "Loss"}' if i == 0 else '',
                showlegend=(i == 0)  # Show legend only for the first trade
            ), row=1, col=1)

            # Plot tp_price and sl_price as narrow lines between entry and exit
            fig.add_trace(go.Scatter(
                x=[entry_time, exit_time], 
                y=[tp_price, tp_price], 
                mode='lines', 
                line=dict(color=tp_p_color, dash='dot', width=2),
                name='Take Profit' if not tp_p_in_legend else '',
                showlegend=not tp_p_in_legend  # Show legend only for the first tp_price
            ), row=1, col=1)
            tp_p_in_legend = True

            fig.add_trace(go.Scatter(
                x=[entry_time, exit_time], 
                y=[sl_price, sl_price], 
                mode='lines', 
                line=dict(color=sl_p_color, dash='dot', width=2),
                name='Stop Loss' if not sl_p_in_legend else '',
                showlegend=not sl_p_in_legend  # Show legend only for the first sl_price
            ), row=1, col=1)
            sl_p_in_legend = True

        # Update layout with titles and axis labels
        fig.update_layout(
            title=f'{symbol} Price and RSI with Trades',
            xaxis_title='Date',
            yaxis_title='Price',
            height=700,
            showlegend=True,
            template="plotly_dark"
        )

        fig.update_yaxes(title_text='RSI', row=2, col=1)
        fig.update_xaxes(rangesl_pider_visible=False)
        
        # Show the interactive plot
        fig.show()

# Example usage (commented out because it requires data):
plot_trades_for_all_symbols(trades_df, mtd.data)


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Calculate performance metrics for each symbol
performance_summary = trades_df.groupby('symbol').agg(
    total_pnl=('pnl', 'sum'),
    max_drawdown=('max_drawdown', 'max'),
    trade_count=('symbol', 'size'),
    win_ratio=('pnl', lambda x: (x > 0).sum() / len(x))  # Win ratio = # of winning trades / total trades
).reset_index()

# Initialize a Plotly figure
fig = go.Figure()

# Create a bar chart for each metric
fig.add_trace(go.Bar(
    x=performance_summary['symbol'],
    y=performance_summary['total_pnl'],
    name='Total PnL',
))

fig.add_trace(go.Bar(
    x=performance_summary['symbol'],
    y=performance_summary['max_drawdown'],
    name='Max Drawdown',
))

fig.add_trace(go.Bar(
    x=performance_summary['symbol'],
    y=performance_summary['trade_count'],
    name='Trade Count',
))

fig.add_trace(go.Bar(
    x=performance_summary['symbol'],
    y=performance_summary['win_ratio'] * 100,  # Convert win ratio to percentage
    name='Win Ratio (%)',
))

# Update layout for better readability
fig.update_layout(
    title='Performance Metrics per Symbol',
    barmode='group',  # Group bars for better comparison
    xaxis_title='Symbol',
    yaxis_title='Value',
    legend_title='Metrics',
    height=600,
    template='plotly_dark'
)

# Show the interactive plot
fig.show()
